In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# SED for Trappist-1
This notebook executes the lines of code presented in Filippazzo et al. (submitted to PASP). Below I generate and analyze an SED for the M8 dwarfs Trappist-1 and Gl 752B. Then I create a catalog of SEDs for analysis.

In [2]:
# Imports
from astropy import units as q
from bokeh.io import output_notebook
from bokeh.plotting import show
from pkg_resources import resource_filename
output_notebook()

/var/folders/n1/5kfqkrz12bz4v70jvvnrtlb80001rh/T/ipykernel_77937/3087237025.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_filename


Loading BokehJS ...

## Initialization
Import the `SED` class and create a Trappist-1 object. The parallax, radius and spectral type are pulled from Simbad when available.

In [3]:
from sedkit import SED
trap1 = SED(name='Trappist-1', substellar=True)

ImportError: cannot import name 'Group' from 'pyparsing' (unknown location)

## Photometry
Add photometry manually...

In [ ]:
trap1.add_photometry('Johnson.V', 18.798, 0.082, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.R', 16.466, 0.065, ref='Costa et al. (2006)')
trap1.add_photometry('Cousins.I', 14.024, 0.115, ref='Costa et al. (2006)')

...and retrieve some from Simbad.

In [202]:
trap1.find_2MASS()
trap1.find_WISE()

[sedkit] 10 records found in '2MASS' using target name 'Trappist-1'
[sedkit] Setting 2MASS.J photometry to 11.354 (0.022) with reference 'II/246/out'
[sedkit] Setting 2MASS.H photometry to 10.718 (0.021) with reference 'II/246/out'
[sedkit] Setting 2MASS.Ks photometry to 10.296 (0.023) with reference 'II/246/out'
[sedkit] 46 records found in 'WISE' using target name 'Trappist-1'
[sedkit] Setting WISE.W1 photometry to 10.068 (0.023) with reference 'II/328/allwise'
[sedkit] Setting WISE.W2 photometry to 9.807 (0.020) with reference 'II/328/allwise'
[sedkit] Setting WISE.W3 photometry to 9.510 (0.044) with reference 'II/328/allwise'
[sedkit] Setting WISE.W4 photometry to 8.565 (nan) with reference 'II/328/allwise'


In [203]:
# Print the photometry table
trap1.photometry[['band', 'eff', 'app_magnitude', 'app_magnitude_unc', 'ref']].pprint()

   band        eff      app_magnitude app_magnitude_unc         ref        
                um                                                         
--------- ------------- ------------- ----------------- -------------------
Johnson.V 0.54638671875          18.8             0.082 Costa et al. (2006)
Cousins.R  0.6357421875         16.47             0.065 Costa et al. (2006)
Cousins.I 0.78271484375         14.02             0.115 Costa et al. (2006)
  2MASS.J  1.2353515625         11.35             0.022          II/246/out
  2MASS.H   1.662109375         10.72             0.021          II/246/out
 2MASS.Ks   2.158203125          10.3             0.023          II/246/out
  WISE.W1   3.353515625         10.07             0.023      II/328/allwise
  WISE.W2     4.6015625         9.805              0.02      II/328/allwise
  WISE.W3       11.5625          9.51             0.044      II/328/allwise
  WISE.W4      22.09375          8.56               nan      II/328/allwise


## Spectra

In [204]:
# Grab spectrum files
nir_spec = resource_filename('sedkit', 'data/Trappist-1_NIR.fits')

# Add the spectra (trimming edge effects off the optical one)
trap1.add_spectrum_file(nir_spec, name='NIR spectrum', wave_units=q.um, flux_units=q.W/q.m**2/q.um, ref='Filippazzo et al. (2015)')

# Print the table
trap1.spectra[['name', 'wave_min', 'wave_max', 'resolution', 'ref']].pprint()

[sedkit] Spectrum 'NIR spectrum' added with reference 'Filippazzo et al. (2015)'
    name       wave_min    wave_max  resolution           ref           
                  um          um                                        
------------ ------------ ---------- ---------- ------------------------
NIR spectrum 0.6533203125 2.56640625        563 Filippazzo et al. (2015)


## Other Data
Set the spectral type.

In [205]:
trap1.spectral_type = 'M8', '2006AJ....132.1234C'

[sedkit] Setting spectral_type to (68.0, 0.5, 'V', None, None) with reference '2006AJ....132.1234C'


Set the age.

In [206]:
trap1.age = 7.6*q.Gyr, 2.2*q.Gyr, '2017ApJ...845..110B'

[sedkit] Setting age to (<Quantity 7.6 Gyr>, <Quantity 2.2 Gyr>) with reference '2017ApJ...845..110B'


Set the radius.

In [207]:
trap1.infer_radius(infer_from='spt', plot=True)

[sedkit] Setting radius to (<Quantity 1.07042746 jupiterRad>, <Quantity 0. jupiterRad>, <Quantity 0. jupiterRad>) with reference '2013ApJS..208....9P'


In [208]:
trap1.radius = 0.121*q.R_sun, 0.003*q.R_sun
trap1.parallax = 82.4*q.mas, 0.8*q.mas

[sedkit] Setting radius to (<Quantity 0.121 solRad>, <Quantity 0.003 solRad>) with reference 'None'
[sedkit] Setting parallax to (<Quantity 82.4 mas>, <Quantity 0.8 mas>) with reference 'None'
[sedkit] Setting distance to (<Quantity 12.13 pc>, <Quantity 0.12 pc>, <Quantity 0.12 pc>) with reference 'None'
[sedkit] Setting interstellar reddening to 0.0 with reference '2018JOSS....3..695M'


## Build SED

In [209]:
trap1.make_sed()
trap1.plot(integral=True)

[sedkit] Setting fbol to (<Quantity 0. erg / (cm2 s)>, <Quantity 0. erg / (cm2 s)>) with reference 'This Work'
[sedkit] Setting Lbol to (<Quantity 0. erg / s>, <Quantity nan erg / s>) with reference 'This Work'
[sedkit] Setting Lbol_sun to (-inf, nan) with reference 'This Work'
[sedkit] Setting mbol to (inf, nan) with reference 'This Work'
[sedkit] Setting Mbol to (inf, nan) with reference 'This Work'
[sedkit] Could not calculate logg without Lbol and age
[sedkit] Setting mass to (<Quantity nan jupiterMass>, <Quantity nan jupiterMass>, <Quantity nan jupiterMass>) with reference '2013ApJS..208....9P'
[sedkit] Setting Teff to (<Quantity 0 K>, <Quantity -9223372036854775808 K>) with reference 'This Work'


Figure(id='15454', ...)

## Fit the data

In [11]:
# Fit to infer spectral type
trap1.fit_spectral_type()

ValueError: unsupported pickle protocol: 5

In [12]:
# Fit to infer fundamental params
from sedkit import BTSettl
trap1.fit_modelgrid(BTSettl())

[sedkit] Best fit BT-Settl spec: 2600.0/5.5/0.0/0.0


## Inferring mass, surface gravity, radius, and Teff
Set the evolutionary model grid and optionally infer fundamental parameters.

In [13]:
trap1.evo_model = 'DUSTY00'

[sedkit] Setting evolutionary model to DUSTY00


In [14]:
trap1.infer_logg(plot=True)

[sedkit] Setting logg to [5.28, 0.0] with reference 'DUSTY00'


In [15]:
trap1.infer_mass(plot=True, mass_units=q.Mjup)

[sedkit] Setting mass to [<Quantity 96.353 jupiterMass>, <Quantity 0.325 jupiterMass>] with reference 'DUSTY00'


## Results
The results of the SED calculations can be accessed anytime with the `results` property.

In [16]:
trap1.results

param,value,unc,units
object,object,object,object
name,Trappist-1,--,--
ra,23.10815788888889,--,deg
dec,-5.041399166666666,--,deg
age,7.6,2.2,Gyr
membership,--,--,--
distance,12.47,0.01,pc
parallax,80.2123,0.0716,mas
SpT,M8V,--,--
spectral_type,68.0,0.5,--


The `plot` method plots the SED, spectra, and photometry by default. The best fits and integral surface used to calculate `Lbol` can be included by setting their arguments to `True`. Click on the elements in the legend to toggle.

In [17]:
plt = trap1.plot(integral=True, best_fit=True)

Check out the references of all the data used in the SED with the `refs` property.

In [18]:
trap1.refs

{'parallax': '2020yCat.1350....0G',
 'distance': '2020yCat.1350....0G',
 'reddening': '2018JOSS....3..695M',
 'spectral_type': '2006AJ....132.1234C',
 'age': '2017ApJ...845..110B',
 'radius': '2021PSJ.....2....1A',
 'fbol': 'This Work',
 'Lbol': 'This Work',
 'Lbol_sun': 'This Work',
 'mbol': 'This Work',
 'Mbol': 'This Work',
 'mass': 'DUSTY00',
 'Teff': 'This Work',
 'logg': 'DUSTY00',
 'photometry': {'Johnson.V': 'Costa et al. (2006)',
  'Cousins.R': 'Costa et al. (2006)',
  'Cousins.I': 'Costa et al. (2006)',
  '2MASS.J': 'II/246/out',
  '2MASS.H': 'II/246/out',
  '2MASS.Ks': 'II/246/out',
  'WISE.W1': 'II/328/allwise',
  'WISE.W2': 'II/328/allwise',
  'WISE.W3': 'II/328/allwise',
  'WISE.W4': 'II/328/allwise'},
 'spectra': {'NIR spectrum': 'Filippazzo et al. (2015)'},
 'best_fit': {'BT-Settl spec': '2014IAUS..299..271A'}}

And look at all the best fit models with the `best_fit` property.

In [19]:
trap1.best_fit

{'BT-Settl spec': {'alpha': 0.0,
  'filepath': '/Users/jfilippazzo/Documents/Modules/sedkit/sedkit/data/models/atmospheric/btsettl/lte026-5.5-0.0a+0.0.BT-NextGen.7.dat.xml',
  'label': '2600.0/5.5/0.0/0.0',
  'logg': 5.5,
  'meta': 0.0,
  'spectrum': array([[3.00001489e-01, 3.02471652e-01, 3.04941815e-01, ...,
          2.49942222e+01, 2.49966923e+01, 2.49991625e+01],
         [           nan, 8.42859707e-21, 9.98488595e-21, ...,
          4.81905016e-19, 4.94476228e-19,            nan]]),
  'teff': 2600.0,
  'gstat': 267801.71504864004,
  'const': 4.2200559361746496e-20,
  'wave_units': Unit("um"),
  'flux_units': Unit("erg / (Angstrom cm2 s)"),
  'fit_to': 'spec',
  'full_model': <sedkit.spectrum.Spectrum at 0x7fac4c864c18>}}